In [ ]:
#use this command to install the required packages 
!pip install -r requirements.txt 

In [ ]:
#install these packages to get the resources you need
import pandas as pd
import numpy as np
import requests
import dask as dd

In [ ]:
%%time 
#here is the link to the file in Google Drive

url_raw = 'https://drive.google.com/file/d/13vj4YxKRs1Tw1VPGhTefBxxKQuC5pRd_/view?usp=sharing'
url_raw_mn = 'https://drive.google.com/file/d/1YRh58UiKWNtxfvYa0IzFF3oYL42iBs7d/view?usp=sharing'

# req = requests.get(url_raw)
# url_content = req.content
# with open('test_data.csv',) as file:
#     df=pd.read_csv(file,delimiter=',')

req_mn = requests.get(url_raw_mn)
url_content_mn = req_mn.content
with open('test_data_mn.csv',) as file_mn:
    df_mn=pd.read_csv(file_mn,delimiter=',')

In [ ]:
#look at the properties of the two datasets 
# print(df.head())
# print(df.shape)


print(df_mn.head())
#print(df_mn.shape)

In [ ]:
df_mn.columns

# Data cleaning for Human Capital Project

First Goal: Create a column that identifies whether or not someone works in tech based upon a) their occupation status and/or b) working for/owning a company with a SIC code in the technology sector (i.e., any org with a code in the office of technology) 

In [ ]:
# Data cleaning for test mn data, but can be motified for larger dataset (do not want to crash my laptop while testing)
##Need to create a binary column that identifies if someone is in tech
df_mn.groupby('vb_occupation')['vb_occupation'].count()

In [ ]:
df_mn['vb_occupation'].unique()

In [ ]:
df_mn.groupby('tb_boah_biz_sic_cd')['tb_boah_biz_sic_cd'].count()
#834 different codes

In [ ]:
df_mn['tb_boah_biz_sic_cd'].astype(int, errors = 'ignore').astype(str).split('.')

In [ ]:
df_mn.groupby('tb_boah_biz_sic_2_cd')['tb_boah_biz_sic_2_cd'].count()
#89 different codes

In [ ]:
# load in file with SIC codes
sic_codes=pd.read_excel('SIC_Codes_5.31.2021.xlsx')
sic_codes.columns

#make a dictionary (First attempt)
sic_dict=sic_codes.to_dict('records')
sic_dict

In [ ]:
# antoher type of dictionary that may be easier to manuver
sic_dict2=sic_codes.set_index('SIC Code').to_dict()['Office']
sic_dict2

In [ ]:
#in the dataframe SIC Code is #.0 so the column needs to be clean
list_SIC_occ=[]
for occ_val in df_mn['tb_boah_biz_sic_cd']:
    clean_val=str(occ_val).split('.')[0]
    list_SIC_occ.append(clean_val)
    
# add column with values

df_mn['SIC_code']=df_mn['tb_boah_biz_sic_cd'].map(sic_dict2)
df_mn.head()
df_mn['SIC_code'].unique()

In [ ]:
#dictionary for occupation
occ_codes=target_smart['Field']=='vb.occupation'
occ_df=target_smart[occ_codes]
occ_df.drop(occ_df.index[[0]],inplace=True)
occ_df.head()

In [ ]:
occ_dict=occ_df.set_index('Field Value').to_dict()['Value Description']

In [ ]:
df_mn['Occupation']=df_mn['vb.occupation'].map(occ_dict)
df_mn['Occupation'].unique()

Second Goal: Create a column with a binary code for “tech_hub_zip9” that identifies all zip codes that are associated with the current tech hubs: New York, Boston, Seattle, San Francisco, and San Jose

In [ ]:
tech_zip=pd.read_excel('Tech_ZipCodes.xlsx')
tech_zipl=tech_zip['Zip Code'].to_list()

tech_zipl


In [ ]:
zip_bin=[]
tech_ziplist=[55428,55975] # this is a test list, but once we finalize a zip code list we can
for zip_code in df_mn['vb_tsmart_zip']:
    if zip_code in tech_ziplist:
        zip_code=1
        zip_bin.append(zip_code)
    else:
        zip_code=0
        zip_bin.append(zip_code)
        
zip_bin

In [ ]:
#create new column with this
df_mn['tech_hub_zip9']=zip_bin

Third Goal: Review coding for education, gender, and race to make sure ACS coding maps to coding in Targetsmart data.

In [ ]:
#uncode the education;gender;race columns with targetsmart data 
#1. load targetsmart data
target_smart=pd.read_excel('targetsmart_data_dictionary_q2_2021.xlsx',sheet_name='Copy of Raw TargetSmart Field V')

In [ ]:
df_mn.columns

In [ ]:
df_mn.columns
target_smart.columns=target_smart.iloc[1]
target_smart.drop(target_smart.index[[0,1]],inplace=True)
target_smart.head()

In [ ]:
#2 dictionary for race
race_codes=target_smart['Field']=='vb.voterbase_race'
race_df=target_smart[race_codes]
race_df.head()

In [ ]:
#3 dictionary for gender
gender_codes=target_smart['Field']=='vb.voterbase_gender'
gender_df=target_smart[gender_codes]
gender_df.head()

In [ ]:
#4 dictionary for education
edu_codes=target_smart['Field']=='vb.education'
edu_df=target_smart[edu_codes]
edu_df.drop(edu_df.index[[0]],inplace=True)
edu_df.head()

In [ ]:
#edu dictionary
edu_dict=edu_df.set_index('Field Value').to_dict()['Value Description']
edu_dict

In [ ]:
#created a categorical column for education
df_mn['Education']=df_mn['vb_education'].map(edu_dict)
df_mn.head()

END of First Data Clean Up 6.6.2021